In [1]:
import os  
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.2 pyspark-shell'  
from pyspark import SparkContext
from pyspark.streaming import StreamingContext  
from pyspark.streaming.kafka import KafkaUtils  
from datetime import datetime

batch_interval = 1
window_length = 12 * batch_interval
frequency = 6 * batch_interval

sc = SparkContext(appName="CTR")
sc.setLogLevel("WARN")
ssc = StreamingContext(sc, batch_interval)

directKafkaStream = KafkaUtils.createDirectStream(ssc, ['topic_click'], {"metadata.broker.list": 'localhost:port'})

In [2]:
lines = directKafkaStream.map(lambda x: x[1])

In [3]:
import re
def mk_int(s):
    s = s.strip()
    return int(s) if s else 0
def tuple_sum(a, b):
    return [a[0]+b[0], a[1]+b[1]]
def tuple_minus(a, b):
    return [a[0]-b[0], a[1]-b[1]]

In [4]:
def insert_into_mysql(group_id, clicks, total):
    import MySQLdb
    conn = MySQLdb.connect(host= "host",
                      user="uid",
                      passwd="pwd",
                      db="db")
    
    try:
        x = conn.cursor()
        x.execute("select yn from abonoff;")
        abonoff = 0
        for row in x:
            abonoff = int(row[0])
        if abonoff == 1:
            if group_id == 1:
                x.execute("""INSERT INTO ctr_4 VALUES (%s,%s,%s,%s,%s,%s,%s)""",(datetime.now(), "test", clicks, total, " ", " ", datetime.now()))
            else:
                x.execute("""INSERT INTO ctr_4 VALUES (%s,%s,%s,%s,%s,%s,%s)""",(datetime.now(), "control", clicks, total, " ", " ", datetime.now()))
        else:
            x.execute("""INSERT INTO ctr_4 VALUES (%s,%s,%s,%s,%s,%s,%s)""",(datetime.now(), "control", clicks, total, " ", " ", datetime.now()))
        conn.commit()
        return (str(group_id) + " " + str(abonoff), 100.0 * clicks / total)
    except Exception, e:
        conn.rollback()
        return (str(e), 100.0 * clicks / total)
    conn.close()
    # return ("ctr = ", 100.0 * clicks / total)

In [6]:
user_info = sc.textFile("s3n://moyang28/UserInfo.tsv")

In [8]:
# user_info = user_info.flatMap(lambda line: line.split("\t")[:])
user_info = user_info.map(lambda line: line.split("\t")[:])
type(user_info)
# user_info.pprint(10)

pyspark.rdd.PipelinedRDD

In [12]:
ctr = lines.map(lambda line: line.split("\t")) 
# ctr.pprint()

In [13]:
ctr = ctr.map(lambda click: (click[0], mk_int(click[-1]))) 
# ctr.pprint()  

In [14]:
ctr = ctr.reduceByKey(lambda a, b: a+b) 
# ctr.pprint()  

In [15]:
ctr = ctr.map(lambda kvpair: (mk_int(kvpair[0])%2, (kvpair[1], 1)))
# ctr.pprint()  

In [16]:
ctr = ctr.reduceByKeyAndWindow(lambda a, b: tuple_sum(a, b), None, 12, 6)
# ctr.pprint()

In [ ]:
ctr = ctr.flatMap(lambda row: insert_into_mysql(row[0], row[1][0], row[1][1]) )
ctr.pprint()

In [ ]:
ssc.start()  
ssc.awaitTermination()

-------------------------------------------
Time: 2017-09-29 21:41:03
-------------------------------------------
0 0
1.09539023277
1 0
0.995625282848

-------------------------------------------
Time: 2017-09-29 21:41:09
-------------------------------------------
0 0
1.11821086262
1 0
1.03222557905

-------------------------------------------
Time: 2017-09-29 21:41:15
-------------------------------------------
0 0
1.07438016529
1 0
1.10766045549

-------------------------------------------
Time: 2017-09-29 21:41:21
-------------------------------------------
0 0
1.18769883351
1 0
1.03934669636

-------------------------------------------
Time: 2017-09-29 21:41:27
-------------------------------------------
0 0
1.1690821256
1 0
0.940950552088

-------------------------------------------
Time: 2017-09-29 21:41:33
-------------------------------------------
0 0
1.08594765175
1 0
1.01363803907

-------------------------------------------
Time: 2017-09-29 21:41:39
-----------------------

In [ ]:
ssc.stop()
sc.stop()